In [48]:
import pandas as pd

In [49]:
url = r"C:\Users\seksatta\Desktop\masterlist.xlsx"

In [50]:
df = pd.read_excel(url, skiprows=6, 
                  parse_dates=True)
df

,No.,Parts and cover page\nreceived date,Actual received parts,Basic investigate\nreceived date,Plan to finished basic investigation date,Plan\nSend\ndate,Actual \nSend\ndate,Plan of Analysis start date,Doc. No.,Production date,...,Report,Final report,Date of remain for analysis\n(Ref. std LT),Control Analysis LT\n(Ref. std LT),LT Usage Workday,Diff from due date (Workday),Checking result\n from Supplier OK/NG,Cause,Part NG,Remark
0,1,2015-03-12 00:00:00,-,-,-,2015-03-12 00:00:00,2015-03-12 00:00:00,-,MKG-15-227,-,...,2015-03-24 00:00:00,-,-,-,-,-,-,-,-,-
1,2,2015-03-12 00:00:00,-,-,-,2015-03-12 00:00:00,2015-03-12 00:00:00,-,MKG-15-228,-,...,2015-03-24 00:00:00,-,-,-,-,-,-,-,-,-
2,3,2015-03-12 00:00:00,-,-,-,2015-03-12 00:00:00,2015-03-12 00:00:00,-,MKG-15-229,-,...,2015-03-24 00:00:00,-,-,-,-,-,-,-,-,-
3,4,2015-03-12 00:00:00,-,-,-,2015-03-12 00:00:00,2015-03-12 00:00:00,-,MKG-15-245,-,...,2015-03-24 00:00:00,-,-,-,-,-,-,-,-,-
4,5,2015-03-12 00:00:00,-,-,-,2015-03-12 00:00:00,2015-03-12 00:00:00,-,MKG-15-246,-,...,2015-03-24 00:00:00,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8597,8598,-,-,-,-,2024-01-09 00:00:00,-,2024-01-10 00:00:00,696,2023-02-14 00:00:00,...,2024-02-06 00:00:00,-,-25,31,55,12,NG,D701 short circuit,Possibility cause dioad damage from surge current,-
8598,8599,-,-,-,-,2024-01-09 00:00:00,-,2024-01-10 00:00:00,697,2023-07-13 00:00:00,...,2024-02-06 00:00:00,-,-25,31,55,12,NG,D701 short circuit,Possibility cause dioad damage from surge current,-
8599,8600,-,-,-,-,2024-01-30 00:00:00,-,2024-01-30 00:00:00,720,2022-10-18 00:00:00,...,2024-02-14 00:00:00,-,-11,31,41,20,NG,R601,R601 part NG possible cause may some kind of c...,-
8600,8601,2024-02-20 00:00:00,-,-,-,2024-02-22 00:00:00,2024-02-21 00:00:00,2024-02-22 00:00:00,709,2023-01-13 00:00:00,...,2024-03-04 00:00:00,-,-15,10,24,3,NG,Wire take out,The lead wires pulled out by appying an abnorm...,-


In [51]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.lower().str.replace("\n","  ")
df.columns = df.columns.str.lower().str.replace(r' ','_')

In [52]:
df

,no.,parts__and_cover_page__received_date,actual_received_parts,basic_investigate__received_date,plan_to_finished_basic_investigation_date,plan__send__date,actual___send__date,plan_of_analysis_start_date,doc._no.,production_date,...,report,final_report,date_of_remain_for_analysis__(ref._std__lt),control_analysis_lt__(ref._std__lt),lt__usage_workday,diff_from_due_date_(workday),checking_result___from_supplier_ok/ng,cause,part_ng,remark
0,1,2015-03-12 00:00:00,-,-,-,2015-03-12 00:00:00,2015-03-12 00:00:00,-,MKG-15-227,-,...,2015-03-24 00:00:00,-,-,-,-,-,-,-,-,-
1,2,2015-03-12 00:00:00,-,-,-,2015-03-12 00:00:00,2015-03-12 00:00:00,-,MKG-15-228,-,...,2015-03-24 00:00:00,-,-,-,-,-,-,-,-,-
2,3,2015-03-12 00:00:00,-,-,-,2015-03-12 00:00:00,2015-03-12 00:00:00,-,MKG-15-229,-,...,2015-03-24 00:00:00,-,-,-,-,-,-,-,-,-
3,4,2015-03-12 00:00:00,-,-,-,2015-03-12 00:00:00,2015-03-12 00:00:00,-,MKG-15-245,-,...,2015-03-24 00:00:00,-,-,-,-,-,-,-,-,-
4,5,2015-03-12 00:00:00,-,-,-,2015-03-12 00:00:00,2015-03-12 00:00:00,-,MKG-15-246,-,...,2015-03-24 00:00:00,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8597,8598,-,-,-,-,2024-01-09 00:00:00,-,2024-01-10 00:00:00,696,2023-02-14 00:00:00,...,2024-02-06 00:00:00,-,-25,31,55,12,NG,D701 short circuit,Possibility cause dioad damage from surge current,-
8598,8599,-,-,-,-,2024-01-09 00:00:00,-,2024-01-10 00:00:00,697,2023-07-13 00:00:00,...,2024-02-06 00:00:00,-,-25,31,55,12,NG,D701 short circuit,Possibility cause dioad damage from surge current,-
8599,8600,-,-,-,-,2024-01-30 00:00:00,-,2024-01-30 00:00:00,720,2022-10-18 00:00:00,...,2024-02-14 00:00:00,-,-11,31,41,20,NG,R601,R601 part NG possible cause may some kind of c...,-
8600,8601,2024-02-20 00:00:00,-,-,-,2024-02-22 00:00:00,2024-02-21 00:00:00,2024-02-22 00:00:00,709,2023-01-13 00:00:00,...,2024-03-04 00:00:00,-,-15,10,24,3,NG,Wire take out,The lead wires pulled out by appying an abnorm...,-


In [53]:
df.problem

0                 Don't operate
1                 Don't operate
2                          Burn
3       Motor cannot turned off
4                 Don't operate
                 ...           
8597          Diode D701 Broken
8598          Diode D701 Broken
8599           LC-14 Error code
8600        connector field off
8601        connector field off
Name: problem, Length: 8602, dtype: object

In [54]:
df['error_code'] = df['problem'].str.extract(r'(\b[A-Z]\d{1})(?<!-)')


0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
8597     D7
8598     D7
8599    NaN
8600    NaN
8601    NaN
Name: error_code, Length: 8602, dtype: object

In [55]:
df['error_code'].value_counts()

error_code
U4    648
L5    482
U2    280
E7    263
C1    200
J9    179
A6    172
L1    164
E2    100
U0     79
H0     66
E5     66
E0     46
F3     42
H6     35
E6     30
E3     22
L9     21
E1     20
F4     20
L8     18
A3     17
A1     17
C4     14
E9     11
H7     11
R1     10
U3     10
L4      9
U1      8
A9      8
A5      7
E8      6
H3      6
C6      6
U9      4
H5      4
U7      4
U5      4
D7      4
E4      3
F6      3
A8      3
H9      2
H8      2
J7      2
R6      2
J6      2
J8      2
J3      1
C3      1
L0      1
C9      1
M1      1
A7      1
U8      1
C5      1
Q1      1
Name: count, dtype: int64